In [1]:
import os
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AzureOpenAI
 
load_dotenv()
 
langfuse = Langfuse()  # Auto-reads LANGFUSE_* env vars
 
AZURE_CLIENT = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-08-01-preview",
)
JUDGE_MODEL = os.getenv("AZURE_OPENAI_MODEL")

In [171]:
import pprint

In [22]:
x = langfuse.api.observations.get_many(trace_id="ac082ab9c427f1ae841cb68282c94748")
# x = langfuse.api.trace.get(trace_id="71c11fa1370bed39e08218f48943afa2")
trace=x.dict()
trace

{'data': [{'id': '9dceb8ec8c681c1a',
   'traceId': 'ac082ab9c427f1ae841cb68282c94748',
   'type': 'AGENT',
   'name': 'rohit',
   'startTime': datetime.datetime(2025, 11, 26, 4, 42, 48, 528000, tzinfo=datetime.timezone.utc),
   'endTime': datetime.datetime(2025, 11, 26, 4, 42, 48, 794000, tzinfo=datetime.timezone.utc),
   'completionStartTime': None,
   'model': None,
   'modelParameters': {},
   'input': {'args': [],
    'kwargs': {'message_request': {'id': 'c7d6a94a-af10-40d8-8915-53cb19e870bb',
      'jsonrpc': '2.0',
      'method': 'message/send',
      'params': {'configuration': None,
       'message': {'contextId': '54c1f9a9-bc18-40ab-a72c-4c40be8c35a2',
        'extensions': None,
        'kind': 'message',
        'messageId': 'c7d6a94a-af10-40d8-8915-53cb19e870bb',
        'metadata': None,
        'parts': [{'kind': 'text',
          'metadata': None,
          'text': 'Provide 10 facts about New Delhi, the capital of India.'}],
        'referenceTaskIds': None,
        'ro

In [23]:
output= None
for obs in trace['data']:
                if obs['type'] == "GENERATION":
                        input = obs['input']['contents'][0]['parts'][0]['text']
                        contents = obs['output']['content']
                        for sub_content in contents['parts']:
                                if sub_content.get('text'):
                                        output = sub_content.get('text')
                        trace_id = obs['traceId']
                        startTime = obs['startTime']
                        model = obs['model']
                        level = trace['meta']['totalItems']
                        environment = obs['environment']
                        input_tokens = obs['usageDetails']['input']
                        output_tokens = obs['usageDetails']['output']
                        total_tokens = obs['usageDetails']['total']
                        latency = obs['latency']
                    

                if output== None:
                    Agents = [obs for obs in trace['data'] if obs.get('type') == 'AGENT']
                    latest_agent = min(Agents, key=lambda x: x['endTime'])
                    # output = latest_agent['output']['result']['artifacts'][0]['parts']['text']
                    artifacts = latest_agent['output']['result']['artifacts']
                    first_artifact = artifacts[0]
                    parts = first_artifact['parts']
                    output = parts[0]['text']

TypeError: 'NoneType' object is not subscriptable

In [24]:
print(input,output,trace_id,startTime,model,level,environment,input_tokens,output_tokens,total_tokens,latency)

tell me the capital of pakistan then tell me 10 facts about that capital None 4c2390b2d103d54b06c697c8a9b4c71c 2025-11-25 13:12:13.225000+00:00 azure/gpt-4o-mini 17 default 617 27 644 14376.0


In [13]:
output = None
for obs in trace['data']:
        if obs['type'] == "GENERATION":
                input = obs['input']['contents'][0]['parts'][0]['text']
                contents = obs['output']['content']
                for sub_content in contents['parts']:
                        if sub_content.get('text'):
                                output = sub_content.get('text')
                trace_id = obs['traceId']
                startTime = obs['startTime']
                model = obs['model']
                level = trace['meta']['totalItems']
                environment = obs['environment']
                input_tokens = obs['usageDetails']['input']
                output_tokens = obs['usageDetails']['output']
                total_tokens = obs['usageDetails']['total']
                latency = obs['latency']

        if output== None:
                Agents = [obs for obs in trace['data'] if obs.get('type') == 'AGENT']
                latest_agent = min(Agents, key=lambda x: x['endTime'])
                # output = latest_agent['output']['result']['artifacts'][0]['parts']['text']
                artifacts = latest_agent['output']['result']['artifacts']
                first_artifact = artifacts[0]
                parts = first_artifact['parts']
                output = parts[0]['text']


In [16]:
print(trace_id,startTime,model,level,environment,input_tokens,output_tokens,total_tokens,latency)

4c2390b2d103d54b06c697c8a9b4c71c 2025-11-25 13:12:13.225000+00:00 azure/gpt-4o-mini 17 default 617 27 644 14376.0


In [14]:
print(input)
print(output)

tell me the capital of pakistan then tell me 10 facts about that capital
The capital of Pakistan is **Islamabad**.

Here are 10 interesting facts about Islamabad:

1. **Purpose-Built Capital**  
Islamabad was specifically designed and constructed in the 1960s to replace Karachi as Pakistan's capital. The city was planned from scratch by Greek architect Constantinos Apostolou Doxiadis.

2. **Geometric Grid Design**  
The city follows a unique grid pattern with sectors numbered and lettered systematically. Each sector is designated by a number and letter combination (like F-6, G-7), making navigation remarkably organized.

3. **Green City by Design**  
Islamabad is one of the world's greenest capitals, with over 70% of the city covered by natural vegetation and parks. It was specifically planned to maintain extensive green spaces.

4. **Twin City Connection**  
Islamabad forms a twin city with neighboring Rawalpindi. While Islamabad serves as the administrative center, Rawalpindi provide

In [10]:
traces = []
for obs in trace['data']:
    if obs['type'] == "AGENT":
        traces.append(obs)

In [166]:
latest_agent = min(traces, key=lambda x: x['endTime'])
pprint.pprint(latest_agent)

{'calculatedInputCost': None,
 'calculatedOutputCost': None,
 'calculatedTotalCost': 0.0,
 'completionStartTime': None,
 'completionTokens': 0,
 'costDetails': {},
 'createdAt': '2025-11-22T14:44:56.533Z',
 'endTime': datetime.datetime(2025, 11, 22, 14, 44, 55, 7000, tzinfo=datetime.timezone.utc),
 'environment': 'default',
 'id': 'dd9cc781aa7ab644',
 'input': {'args': [],
           'kwargs': {'message_request': {'id': '27a1ec3f-3e7c-4410-8318-503639adc462',
                                          'jsonrpc': '2.0',
                                          'method': 'message/send',
                                          'params': {'configuration': None,
                                                     'message': {'contextId': '52bf3458-b345-430d-82cd-136848af21c6',
                                                                 'extensions': None,
                                                                 'kind': 'message',
                                             

In [168]:
output = latest_agent['output']['result']['artifacts'][0]['parts']['text']


TypeError: list indices must be integers or slices, not str

In [167]:
# Sort the agent observations by endTime (ascending), then take the last one

artifacts = latest_agent['output']['result']['artifacts']
first_artifact = artifacts[0]                    # usually only one
parts = first_artifact['parts']
output_text = parts[0]['text']
pprint.pprint(output)

'The manager of the Marketing department is Alejandro Brender.'


In [154]:
pprint.pprint(traces)

[{'calculatedInputCost': None,
  'calculatedOutputCost': None,
  'calculatedTotalCost': 0.0,
  'completionStartTime': None,
  'completionTokens': 0,
  'costDetails': {},
  'createdAt': '2025-11-22T14:45:06.873Z',
  'endTime': datetime.datetime(2025, 11, 22, 14, 45, 5, 728000, tzinfo=datetime.timezone.utc),
  'environment': 'default',
  'id': 'c8dc3cabb5c8a4d1',
  'input': {'args': [],
            'kwargs': {'message_request': {'id': '632f598c-172c-4a11-8dc2-33946b223ad2',
                                           'jsonrpc': '2.0',
                                           'method': 'message/send',
                                           'params': {'configuration': None,
                                                      'message': {'contextId': '082ade9b-61eb-4d66-92ca-1d7ef80df020',
                                                                  'extensions': None,
                                                                  'kind': 'message',
                          

In [112]:
INPUT_COST = 0.15 / 1000000
OUTPUT_COST = 0.60 / 1000000

def calculate_cost(input_tokens, output_tokens):
    input_cost = input_tokens * INPUT_COST
    output_cost = output_tokens * OUTPUT_COST
    total_cost = input_cost + output_cost

    return input_cost, output_cost, total_cost

In [113]:
cost = calculate_cost(input_tokens, output_tokens)

In [119]:
print(cost[1])

7.8e-06


In [128]:
pprint.pprint(id)
pprint.pprint(input)
pprint.pprint(startTime)
pprint.pprint(model)
pprint.pprint(level)
pprint.pprint(environment)
pprint.pprint(input_tokens)
pprint.pprint(output_tokens)
pprint.pprint(total_tokens)

'6f338e14efc427b291ecb6fce691bfae'
'what is the capital of Pakistan and write 10 lines about it.'
datetime.datetime(2025, 11, 20, 9, 28, 20, 473000, tzinfo=datetime.timezone.utc)
'azure/gpt-4o-mini'
7
'default'
921
38
959
